<a href="https://colab.research.google.com/github/saffarizadeh/INSY4054/blob/main/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="http://saffarizadeh.com/Logo.png" width="300px"/>

# *INSY 4054: Emerging Technologies*

# **Trained Models and Transfer Learning**

Instructor: Dr. Kambiz Saffarizadeh

---

Source: https://tfhub.dev/google/nnlm-en-dim50/2

Read the complete tutorial: https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow_datasets as tfds

We first use `tensorflow_datasets` to load the imdb_reviews dataset.
`tensorflow_datasets` loads large datasets in a specific way that works well with TensorFlow models. The data loaded using this method is not in numpy array format. Instead it is a TensorFlow Dataset (see https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [ ]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

We load the pretrained model and extend it using a Sequential model to classify the input into two classes (positive vs. negative sentiment).
Note that we set the model as `trainable` which means all parameters can be retrained. Since those parameters already have some weights and biases that work well, it is likely that the final weights and biases are close to what we already have in this part of the model.

In [4]:
model = tf.keras.models.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2", input_shape=[], dtype=tf.string, trainable=True),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Since the dataset we are using is large, we can choose the batch size we want to use to feed the data into our model. Before batching the data, we can shuffle it. To have the same shuffling outcome everytime we run this code on our machine, we can set a seed for the random shuffle. Here we use `10000`. We can use any number we want.

In [7]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 6s 94ms/step - loss: 0.7195 - accuracy: 0.4935 - val_loss: 0.6675 - val_accuracy: 0.6405
Epoch 2/10
30/30 [==============================] - 3s 84ms/step - loss: 0.6455 - accuracy: 0.6774 - val_loss: 0.5907 - val_accuracy: 0.7493
Epoch 3/10
30/30 [==============================] - 3s 84ms/step - loss: 0.5394 - accuracy: 0.7998 - val_loss: 0.4766 - val_accuracy: 0.8144
Epoch 4/10
30/30 [==============================] - 3s 85ms/step - loss: 0.3964 - accuracy: 0.8742 - val_loss: 0.3776 - val_accuracy: 0.8524
Epoch 5/10
30/30 [==============================] - 3s 84ms/step - loss: 0.2720 - accuracy: 0.9210 - val_loss: 0.3303 - val_accuracy: 0.8645
Epoch 6/10
30/30 [==============================] - 3s 83ms/step - loss: 0.1964 - accuracy: 0.9450 - val_loss: 0.3114 - val_accuracy: 0.8717
Epoch 7/10
30/30 [==============================] - 3s 85ms/step - loss: 0.1421 - accuracy: 0.9618 - val_loss: 0.3052 - val_accuracy: 0.8738
Epoch 8/10
30

In [8]:
model.evaluate(test_data.batch(512), verbose=2)

49/49 - 2s - loss: 0.3598 - accuracy: 0.8558


[0.3597661852836609, 0.8557599782943726]

In [9]:
new_reviews = ["This was a great movie", "This was a very bad movie"]

In [10]:
new_reviews_array = np.array(new_reviews)

In [11]:
model(new_reviews_array)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.98168576],
       [0.14568348]], dtype=float32)>